In [ ]:
#run this code
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#run this code
!pip install happytransformer
!pip install lime
!pip install shap

In [ ]:
# #tokenizing
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# train_tokens = tokenizer(list(train['TEXT']), padding=True, truncation=True, return_tensors='pt', max_length=500)
# validation_tokens = tokenizer(list(validation['TEXT']), padding=True, truncation=True, return_tensors='pt', max_length=500)
# test_tokens = tokenizer(list(test['TEXT']), padding=True, truncation=True, return_tensors='pt', max_length=500)

In [ ]:
# # Save tokenized data
# torch.save(train_tokens, 'train_tokens.pt')
# torch.save(validation_tokens, 'validation_tokens.pt')
# torch.save(test_tokens, 'test_tokens.pt')

In [ ]:
# # Load tokenized data
# loaded_train_tokens = torch.load('train_tokens.pt')
# loaded_test_tokens = torch.load('test_tokens.pt')

In [ ]:
# #Model
# from transformers import AutoModelForSequenceClassification, AdamW

# model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=2)

In [ ]:
# #Training
# from torch.utils.data import DataLoader, TensorDataset
# from torch.optim import AdamW


# learning_rate = 1e-3
# batch_size = 10
# epochs = 9

# train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], torch.tensor(list(train['ICD9_CODE_encoded'])))
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# optimizer = AdamW(model.parameters(), lr=learning_rate)

# for epoch in range(epochs):
#     for batch in train_loader:
#         optimizer.zero_grad()
#         inputs, attention_mask, labels = batch
#         outputs = model(inputs, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

In [ ]:
# #Evaluation
# from sklearn.metrics import classification_report

# with torch.no_grad():
#     model.eval()
#     predictions = model(test_tokens['input_ids'], attention_mask=test_tokens['attention_mask']).logits.argmax(dim=1)

# print(classification_report(test_labels, predictions))


In [ ]:
##using happytransformer

# from happytransformer import HappyTextClassification, TCTrainArgs
# from torchinfo import summary
# # --------------------------------------#
# happy_tc = HappyTextClassification(model_type="BERT",
#                                    model_name="medicalai/ClinicalBERT",
#                                    num_labels=2)

# args = TCTrainArgs(learning_rate = 1e-5, batch_size = 32, num_train_epochs = 9, save_steps= 0.1, output_dir = "/content/drive/MyDrive/models_clinicalBERT")
# training = happy_tc.train("/content/drive/MyDrive/2_whole_dataset_train_validate_df_100.csv", args=args)

In [ ]:
# # #using happytransformer
# # #using BioBERT
# from happytransformer import HappyTextClassification, TCTrainArgs
# from torchinfo import summary
# # --------------------------------------#
# happy_tc = HappyTextClassification(model_type="BioBERT", model_name="emilyalsentzer/Bio_ClinicalBERT",
#                                  num_labels=2)

# # Train the model
# args = TCTrainArgs(learning_rate = 1e-6, batch_size = 32, num_train_epochs = 5, save_steps= 0.1, output_dir = "/content/drive/MyDrive/models_BIOBERT")
# happy_tc.train('/content/drive/MyDrive/whole_dataset/2_preprocessed_train_validate_df_60.csv', args=args)
# happy_tc.save("finetuned_model/")

In [ ]:
#run this code
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import re
import torch
import shap

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/whole_dataset/final_dataset_5.csv', low_memory=False)

In [ ]:
df.head()

,Unnamed: 0,SUBJECT_ID,ICD9_CODE,TEXT,GENDER,clean_len
0,958,112,41401,Admission Date: [**2194-6-13**] Dischar...,M,948
1,959,112,41401,Admission Date: [**2196-9-27**] ...,M,881
2,965,114,41401,Admission Date: [**2146-8-29**] Dischar...,M,558
3,980,115,4280,Admission Date: [**2194-10-16**] ...,F,1552
4,982,115,4019,Admission Date: [**2194-10-16**] ...,F,1552


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.drop(['Unnamed: 0','GENDER', 'SUBJECT_ID','clean_len'],axis=1, inplace=True)

In [ ]:
df.rename(columns={'ICD9_CODE': 'label', 'TEXT': 'text'}, inplace=True)

In [ ]:
#Splitting
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])


#encoding label
encoder = LabelEncoder()
#train_set
train_df['label'] = encoder.fit_transform(train_df['label'])
#test_set
test_df['label'] = encoder.transform(test_df['label'])

In [ ]:
train_df = train_df[['text', 'label']]
test_df = test_df[['text', 'label']]

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75102 entries, 71361 to 19666
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    75102 non-null  object
 1   label   75102 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ MB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18776 entries, 69532 to 70322
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18776 non-null  object
 1   label   18776 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 440.1+ KB


In [ ]:
# train_df.to_csv('/content/drive/MyDrive/whole_dataset/5_train_whole_dataset_longformer.csv')
# test_df.to_csv('/content/drive/MyDrive/whole_dataset/5_test_whole_dataset_longformer.csv')

In [ ]:
# #using happytransformer
# #using RoBERTa(longformer)
# from happytransformer import HappyTextClassification, TCTrainArgs
# # --------------------------------------#

# happy_tc = HappyTextClassification(model_type="LONGFORMER", model_name="yikuan8/Clinical-Longformer",
#                                  num_labels=5)

# # Train the model
# args = TCTrainArgs(learning_rate = 1e-6, batch_size = 4, num_train_epochs = 20, save_steps= 0.1, output_dir = "/content/drive/MyDrive/models_longformer")
# happy_tc.train("/content/drive/MyDrive/whole_dataset/5_train_whole_dataset_longformer.csv", args=args)
# happy_tc.save("finetuned_longformer_model/")


In [ ]:
#run this code
#load model
from happytransformer import HappyTextClassification

model = HappyTextClassification(model_type="BioBERT", model_name="/content/drive/MyDrive/finetuned_model")

In [ ]:
# # clipping approach
# from happytransformer import HappyTextClassification
# import re
# from collections import Counter, namedtuple

# # Define a simple namedtuple to represent the classification result
# TextClassificationResult = namedtuple("TextClassificationResult", ["label", "score"])

# # Load your pre-trained model and tokenizer
# model = HappyTextClassification(model_type="BioBERT", model_name="/content/drive/MyDrive/finetuned_model")
# tokenizer = model.tokenizer

# def prep_simple(pdf_read):
#     # Your existing preprocessing code
#     pdf_modified = re.sub(' +', ' ', pdf_read)
#     pdf_modified = pdf_modified.lower()
#     pdf_modified = pdf_modified.replace("\n", " ")
#     pdf_modified = re.sub(r'\[.*?\]', '', pdf_modified)
#     pdf_modified = re.sub(' +', ' ', pdf_modified)
#     return pdf_modified.strip()

# def tokenize_and_predict(text):
#     # Apply preprocessing
#     preprocessed_text = prep_simple(text)

#     # Define the maximum token limit
#     max_token_limit = 512

#     # Preprocess the text by splitting it into chunks
#     chunks = re.findall(r"\S+\s*", preprocessed_text)
#     chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

#     # Initialize predictions
#     predictions = []

#     # Process each chunk
#     for chunk in chunks:
#         # Tokenize the chunk
#         # returns a list with every word or punctuation in a single string
#         tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(chunk)))

#         # Ensure the chunk does not exceed the token limit
#         if len(tokens) <= max_token_limit:
#             # Make prediction for the chunk
#             prediction = model.classify_text(chunk)
#             predictions.append(prediction)
#         else:
#             print(f"Warning: Chunk exceeds token limit and will be skipped:\n{chunk}")

#     # Aggregate predictions (simple majority vote based on label and score)
#     aggregated_prediction = aggregate_predictions(predictions)

#     return aggregated_prediction

# def aggregate_predictions(predictions):
#     # Count occurrences of each label in the predictions
#     label_counts = Counter([result.label for result in predictions])

#     # Find the label with the maximum count
#     majority_label = max(label_counts, key=label_counts.get)

#     # Find the average score for the majority label
#     majority_score = sum([result.score for result in predictions if result.label == majority_label]) / label_counts[majority_label]

#     # Create a TextClassificationResult object for the aggregated prediction
#     aggregated_prediction = TextClassificationResult(label=majority_label, score=majority_score)

#     return aggregated_prediction

# #predict text
# #label_0 = ICD_CODE(4280)
# #label_1 = ICD_CODE(41401)
# text_to_predict = text
# final_prediction = tokenize_and_predict(text_to_predict)
# #print label
# if final_prediction.label == 'LABEL_0':
#   print('ICD_CODE = 4280')
# elif final_prediction.label == 'LABEL_1':
#   print('ICD_CODE = 41401')
# #print score
# print(f"accuracy: {final_prediction.score:.2f}")
# #print(final_prediction.label)  # Access the label of the aggregated prediction
# #print(final_prediction.score)  # Access the score of the aggregated prediction

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


ICD_CODE = 4280
accuracy: 0.58


In [ ]:
#explainable AI
explainer = shap.TreeExplainer(your_trained_model)
shap_values = explainer.shap_values(your_data)
shap.summary_plot(shap_values, your_data)
shap.force_plot(explainer.expected_value, shap_values[0], your_data.iloc[0, :])

<Logger happytransformer.happy_transformer (WARNING)>

In [ ]:
## run this code
from happytransformer import HappyTextClassification
from lime.lime_text import LimeTextExplainer
import re
from collections import Counter, namedtuple

# Define a simple namedtuple to represent the classification result
TextClassificationResult = namedtuple("TextClassificationResult", ["label", "score"])

# Load your pre-trained model and tokenizer
model = HappyTextClassification(model_type="BioBERT", model_name="/content/drive/MyDrive/finetuned_model")
tokenizer = model.tokenizer

def prep_simple(pdf_read):
    # Your existing preprocessing code
    pdf_modified = re.sub(' +', ' ', str(pdf_read))
    pdf_modified = pdf_modified.lower()
    pdf_modified = pdf_modified.replace("\n", " ")
    pdf_modified = re.sub(r'\[.*?\]', '', pdf_modified)
    pdf_modified = re.sub(' +', ' ', pdf_modified)
    return pdf_modified.strip()

def tokenize_and_predict(text):
    # Apply preprocessing
    preprocessed_text = prep_simple(text)

    # Define the maximum token limit
    max_token_limit = 512

    # Preprocess the text by splitting it into chunks
    chunks = re.findall(r"\S+\s*", preprocessed_text)
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

    # Initialize predictions
    predictions = []

    # Process each chunk
    for chunk in chunks:
        # Tokenize the chunk
        # returns a list with every word or punctuation in a single string
        tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(chunk)))

        # Ensure the chunk does not exceed the token limit
        if len(tokens) <= max_token_limit:
            # Make prediction for the chunk
            prediction = model.classify_text(chunk)
            predictions.append(prediction)
        else:
            print(f"Warning: Chunk exceeds token limit and will be skipped:\n{chunk}")

    # Aggregate predictions (simple majority vote based on label and score)
    aggregated_prediction = aggregate_predictions(predictions)

    return aggregated_prediction

def aggregate_predictions(predictions):
    # Count occurrences of each label in the predictions
    label_counts = Counter([result.label for result in predictions])

    # Find the label with the maximum count
    majority_label = max(label_counts, key=label_counts.get)

    # Find the average score for the majority label
    majority_score = sum([result.score for result in predictions if result.label == majority_label]) / label_counts[majority_label]

    # Create a TextClassificationResult object for the aggregated prediction
    aggregated_prediction = TextClassificationResult(label=majority_label, score=majority_score)

    return aggregated_prediction


############################### explain text function
##### tokenize and predict--> (classifier function(model used for classifing text if it is ICD code 4280 or 41401))
def explain_text(text):
    # Create LIME explainer
    explainer = LimeTextExplainer(class_names=[0, 1])

    # Explain the model's prediction for the given text
    explanation = explainer.explain_instance(text, tokenize_and_predict, num_features=30)

    # Print the explanation
    print(explanation.as_list())

# Example usage
text = """Unit No:  [**Numeric Identifier 56787**]\nAdmission Date: [**2119-11-14**]\nDischarge Date: [**2119-12-1**]\nDate of Birth:  [**2058-6-3**]\nSex:  M\nService:  CSU\n\n\nHISTORY OF PRESENT ILLNESS:  Father [**Name (NI) **] is a 61-year-old\nman, with known CAD, status post coronary artery bypass graft\non [**2119-10-31**] with a LIMA to the LAD, saphenous vein graft to\nOM1, saphenous vein graft to D1, and saphenous vein graft to\nPDA.  The patient was discharged home on [**11-5**], and returns\non the day of admission complaining of sternal drainage x\nseveral days with increasing amounts on the day of admission.\nThe patient denies fever, chills, nausea, vomiting, or\nmalaise.\n\nPAST MEDICAL HISTORY:  CAD, status post CABG with an EF of 20\npercent.\n\nDiabetes mellitus, currently insulin dependent.\n\nHypercholesterolemia.\n\nGERD.\n\nALLERGIES:  None.\n\nMEDS ON ADMISSION:\n1. Colace 100 mg [**Hospital1 **].\n2. Aspirin 81 mg once daily.\n3. Plavix 75 mg once daily.\n4. Carvedilol 6.25 mg [**Hospital1 **].\n5. Simvastatin 40 mg once daily.\n6. Lasix 40 mg [**Hospital1 **].\n7. Lantus insulin 45 units q pm.\n8. Percocet 5/325, 1-2 tabs q 4 h prn.\n\n\nLABS ON ADMISSION:  White count 18.6, hematocrit 33.9,\nplatelets 893, PT 17.5, PTT 24, INR 1.1, sodium 139,\npotassium 4.2, chloride 101, CO2 25, BUN 14, creatinine 0.9,\nglucose 246.  Chest x-ray shows cardiomegaly with left-sided\neffusion with atelectasis, multiple displaced wires.  EKG:\nSinus rhythm with a rate of 100, Q's in III and AVF,\nnonspecific ST changes with poor R wave progression.\n\nPHYSICAL EXAM:  Temperature 103, heart rate 116--sinus\ntachycardia, blood pressure 100/47, respiratory rate 30, O2\nsat 97 percent on 2 liters nasal prongs.  Neuro:  Alert and\noriented x 3, moves all extremities, follows commands,\nnonfocal exam.  Respiratory:  Clear to auscultation with a\nsucking chest wound.  Cardiovascular:  Regular rate and\nrhythm.  Sternum with surrounding erythema of about 10 cm,\nwith a positive click.  Small draining hole in midincision\nwith milky serous drainage.  Staples remain in place.\nAbdomen is soft, nontender, nondistended with normoactive\nbowel sounds.  Extremities are warm and well-perfused with no\nedema.  Right calf with a healing wound and minimal erythema.\nLeft knee with an endoscopic site that is healing, open to\nair, clean and dry.\n\nHOSPITAL COURSE:  The patient was admitted to the\nCardiothoracic Intensive Care Unit.  He was begun on\nvancomycin 1 gm q 12 h, as well as levofloxacin 500 mg once\ndaily.  He was typed and screened and kept NPO for\nmediastinal exploration plus/minus a flap closure.\n\nOn hospital day 2, the patient was brought to the operating\nroom.  Please see the OR report for full details.  In\nsummary, the patient had a sternal exploration and\ndebridement.  He tolerated the operation well and was\nreturned to the Cardiothoracic Intensive Care Unit intubated\nand sedated with an open chest wound.  Plastic surgery was\nalso following the patient.\n\nThe patient did well in the immediate postoperative period.\nHis anesthesia was reversed.  He was weaned from the\nventilator and successfully extubated.  Several hours\nfollowing extubation, the patient was found to be in acute\nrespiratory distress and was emergently reintubated.  From\nthat point forward, he was kept sedated and ventilated\nawaiting plastics follow-up for flap closure.\n\nOn the [**11-19**], the patient returned to the operating\nroom.  Please see the OR report for full details.  In\nsummary, the patient was brought to the operating room by the\nplastic surgery service for pectoral advancement with an\nomentum flap.  He tolerated the operation well and was\nreturned to the Cardiothoracic Intensive Care Unit.  The\npatient remained intubated following his surgery.  However,\nhis sedation was minimized to allow the patient to\noverbreathe the ventilator.  During that period, the patient\nhad several episodes of coughing which led to a dehiscence of\nhis abdominal incision, and on the [**11-20**] the patient\nagain returned to the operating room for re-exploration and\nclosure of the fascia of his abdominal wound.  He tolerated\nthis surgery well also and following that returned to the\nCardiothoracic Intensive Care Unit, again ventilated and\nsedated.  The patient remained ventilated and sedated for the\nnext several days in an attempt to give the wound a chance to\nheal.\n\nUltimately, the patient was successfully extubated on the [**11-24**].  However, he stayed in the Cardiothoracic\nIntensive Care Unit following extubation for close pulmonary\nmonitoring.  It should be noted that during the patient's ICU\ncourse, he had several intermittent episodes of atrial\nfibrillation for which he was begun on amiodarone, as well as\nheparin and ultimately Coumadin for anticoagulation.  The\npatient did well over the next several days, and ultimately\nwas transferred to the floor on [**11-28**], hospital day 15,\npostoperative day 13.  At that point, a PICC line was placed\nfor long-term antibiotic coverage.\n\nOver the next several days, the patient's activity level was\nincreased with the assistance of the nursing and the physical\ntherapy staff.  His antibiotic coverage was continued.  His\nanticoagulation was transitioned from intravenous to oral.\nFinally, on the [**12-1**], the patient's final [**Location (un) 1661**]-\n[**Location (un) 1662**] drain was removed from his chest, and it was decided\nthat he was stable and ready to be transferred to\nrehabilitation for long-term antibiotic coverage, as well as\nglucose control.\n\nAt that time, the patient's physical exam was as follows:\nVital signs:  Temperature 98.4, heart rate 82--sinus rhythm,\nblood pressure 113/66, respiratory rate 18, O2 sat 95 percent\non room air, weight day of dictation 106.6 kg, preoperatively\n100 kg.  Lab data:  PT 17.1, INR 1.9, sodium 139, potassium\n3.7, chloride 100, bicarb 27, BUN 11, creatinine 0.9, glucose\n149, white count 9.1, hematocrit 28.4, platelets 830.\nPhysical exam - Neurologically:  Alert and oriented x 3,\nnonfocal exam.  Pulmonary:  Clear to auscultation\nbilaterally.  Cardiac:  Regular rate and rhythm, S1, S2.\nSternum:  Incision with staples, clean and dry.  No erythema\nor drainage.  Abdomen was soft, nontender, nondistended with\nnormoactive bowel sounds.  Abdominal incision with staples,\nalso clean and dry.  Extremities were warm with no edema.\nRight saphenous vein graft harvest site was healing well,\nopen to air, clean and dry.\n\nCONDITION ON DISCHARGE:  Good.\n\nDISCHARGE DIAGNOSES:  Coronary artery disease, status post\ncoronary artery bypass grafting complicated by sternal\ninfection requiring sternal debridement and flap closure.\n\nDiabetes mellitus.\n\nHypercholesterolemia.\n\nGastroesophageal reflux disease.\n\nFOLLOW UP:  Follow-up with Dr. [**Last Name (STitle) 13797**] with plastic surgery\nservice in 1 week.  He is to call for an appointment at [**Telephone/Fax (1) 56789**].  He is also to have follow-up with Dr. [**Last Name (STitle) **] in 4\nweeks.  The patient is also to call for that appointment; the\nnumber is [**Telephone/Fax (1) 170**].\n\nDISCHARGE MEDICATIONS:\n1. Ranitidine 150 mg [**Hospital1 **].\n2. Simvastatin 40 mg once daily.\n3. Ferrous sulfate 325 mg once daily.\n4. Ascorbic acid 500 mg [**Hospital1 **].\n5. Zinc sulfate 220 mg once daily.\n6. Aspirin 81 mg once daily.\n7. Erythromycin ophthalmic ointment [**Hospital1 **].\n8. Colace 100 mg [**Hospital1 **].\n9. Metoprolol XL 100 mg once daily.\n10.Glargine 24 units q at bedtime.\n11.Humalog insulin sliding scale q ac and at bedtime.\n12.Lasix 20 mg once daily.\n13.Potassium chloride 20 mEq once daily.\n14.Amiodarone 400 mg [**Hospital1 **] x 1 week, then 400 mg once daily x 1\nweek, then 200 mg once daily.\n15.Oxacillin 2 grams q 4 h through [**12-28**].\n16.Warfarin as directed to maintain a target INR of 2 to 2.5.\nThe patient's warfarin doses starting with 4 days ago - 3 mg,\n5 mg, 5 mg, 5 mg.  The patient is to receive 4 mg on the [**2032-11-29**].Albuterol 2 puffs qid prn.\n\nDISPOSITION:  The patient is to be discharged to\nrehabilitation.\n\n\n\n                        [**First Name11 (Name Pattern1) **] [**Last Name (NamePattern1) 1714**], [**MD Number(1) 1715**]\n\nDictated By:[**Last Name (NamePattern4) 1718**]\nMEDQUIST36\nD:  [**2119-12-1**] 13:31:57\nT:  [**2119-12-1**] 14:15:12\nJob#:  [**Job Number 56790**]\n\n\n\n"""
final_prediction = tokenize_and_predict(text)

# Print label and score
if final_prediction.label == 'LABEL_0':
    prediction_label = 0
    print('ICD_CODE = 4280')
elif final_prediction.label == 'LABEL_1':
    prediction_label = 1
    print('ICD_CODE = 41401')

print(f"Score: {final_prediction.score:.2f}")

# Explain the model's prediction
explain_text(text)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


ICD_CODE = 4280
Accuracy: 0.58


  2%|▏         | 3784/211857 [59:49<54:49:49,  1.05it/s]


KeyboardInterrupt: 

In [ ]:
text = """Unit No:  [**Numeric Identifier 56787**]\nAdmission Date: [**2119-11-14**]\nDischarge Date: [**2119-12-1**]\nDate of Birth:  [**2058-6-3**]\nSex:  M\nService:  CSU\n\n\nHISTORY OF PRESENT ILLNESS:  Father [**Name (NI) **] is a 61-year-old\nman, with known CAD, status post coronary artery bypass graft\non [**2119-10-31**] with a LIMA to the LAD, saphenous vein graft to\nOM1, saphenous vein graft to D1, and saphenous vein graft to\nPDA.  The patient was discharged home on [**11-5**], and returns\non the day of admission complaining of sternal drainage x\nseveral days with increasing amounts on the day of admission.\nThe patient denies fever, chills, nausea, vomiting, or\nmalaise.\n\nPAST MEDICAL HISTORY:  CAD, status post CABG with an EF of 20\npercent.\n\nDiabetes mellitus, currently insulin dependent.\n\nHypercholesterolemia.\n\nGERD.\n\nALLERGIES:  None.\n\nMEDS ON ADMISSION:\n1. Colace 100 mg [**Hospital1 **].\n2. Aspirin 81 mg once daily.\n3. Plavix 75 mg once daily.\n4. Carvedilol 6.25 mg [**Hospital1 **].\n5. Simvastatin 40 mg once daily.\n6. Lasix 40 mg [**Hospital1 **].\n7. Lantus insulin 45 units q pm.\n8. Percocet 5/325, 1-2 tabs q 4 h prn.\n\n\nLABS ON ADMISSION:  White count 18.6, hematocrit 33.9,\nplatelets 893, PT 17.5, PTT 24, INR 1.1, sodium 139,\npotassium 4.2, chloride 101, CO2 25, BUN 14, creatinine 0.9,\nglucose 246.  Chest x-ray shows cardiomegaly with left-sided\neffusion with atelectasis, multiple displaced wires.  EKG:\nSinus rhythm with a rate of 100, Q's in III and AVF,\nnonspecific ST changes with poor R wave progression.\n\nPHYSICAL EXAM:  Temperature 103, heart rate 116--sinus\ntachycardia, blood pressure 100/47, respiratory rate 30, O2\nsat 97 percent on 2 liters nasal prongs.  Neuro:  Alert and\noriented x 3, moves all extremities, follows commands,\nnonfocal exam.  Respiratory:  Clear to auscultation with a\nsucking chest wound.  Cardiovascular:  Regular rate and\nrhythm.  Sternum with surrounding erythema of about 10 cm,\nwith a positive click.  Small draining hole in midincision\nwith milky serous drainage.  Staples remain in place.\nAbdomen is soft, nontender, nondistended with normoactive\nbowel sounds.  Extremities are warm and well-perfused with no\nedema.  Right calf with a healing wound and minimal erythema.\nLeft knee with an endoscopic site that is healing, open to\nair, clean and dry.\n\nHOSPITAL COURSE:  The patient was admitted to the\nCardiothoracic Intensive Care Unit.  He was begun on\nvancomycin 1 gm q 12 h, as well as levofloxacin 500 mg once\ndaily.  He was typed and screened and kept NPO for\nmediastinal exploration plus/minus a flap closure.\n\nOn hospital day 2, the patient was brought to the operating\nroom.  Please see the OR report for full details.  In\nsummary, the patient had a sternal exploration and\ndebridement.  He tolerated the operation well and was\nreturned to the Cardiothoracic Intensive Care Unit intubated\nand sedated with an open chest wound.  Plastic surgery was\nalso following the patient.\n\nThe patient did well in the immediate postoperative period.\nHis anesthesia was reversed.  He was weaned from the\nventilator and successfully extubated.  Several hours\nfollowing extubation, the patient was found to be in acute\nrespiratory distress and was emergently reintubated.  From\nthat point forward, he was kept sedated and ventilated\nawaiting plastics follow-up for flap closure.\n\nOn the [**11-19**], the patient returned to the operating\nroom.  Please see the OR report for full details.  In\nsummary, the patient was brought to the operating room by the\nplastic surgery service for pectoral advancement with an\nomentum flap.  He tolerated the operation well and was\nreturned to the Cardiothoracic Intensive Care Unit.  The\npatient remained intubated following his surgery.  However,\nhis sedation was minimized to allow the patient to\noverbreathe the ventilator.  During that period, the patient\nhad several episodes of coughing which led to a dehiscence of\nhis abdominal incision, and on the [**11-20**] the patient\nagain returned to the operating room for re-exploration and\nclosure of the fascia of his abdominal wound.  He tolerated\nthis surgery well also and following that returned to the\nCardiothoracic Intensive Care Unit, again ventilated and\nsedated.  The patient remained ventilated and sedated for the\nnext several days in an attempt to give the wound a chance to\nheal.\n\nUltimately, the patient was successfully extubated on the [**11-24**].  However, he stayed in the Cardiothoracic\nIntensive Care Unit following extubation for close pulmonary\nmonitoring.  It should be noted that during the patient's ICU\ncourse, he had several intermittent episodes of atrial\nfibrillation for which he was begun on amiodarone, as well as\nheparin and ultimately Coumadin for anticoagulation.  The\npatient did well over the next several days, and ultimately\nwas transferred to the floor on [**11-28**], hospital day 15,\npostoperative day 13.  At that point, a PICC line was placed\nfor long-term antibiotic coverage.\n\nOver the next several days, the patient's activity level was\nincreased with the assistance of the nursing and the physical\ntherapy staff.  His antibiotic coverage was continued.  His\nanticoagulation was transitioned from intravenous to oral.\nFinally, on the [**12-1**], the patient's final [**Location (un) 1661**]-\n[**Location (un) 1662**] drain was removed from his chest, and it was decided\nthat he was stable and ready to be transferred to\nrehabilitation for long-term antibiotic coverage, as well as\nglucose control.\n\nAt that time, the patient's physical exam was as follows:\nVital signs:  Temperature 98.4, heart rate 82--sinus rhythm,\nblood pressure 113/66, respiratory rate 18, O2 sat 95 percent\non room air, weight day of dictation 106.6 kg, preoperatively\n100 kg.  Lab data:  PT 17.1, INR 1.9, sodium 139, potassium\n3.7, chloride 100, bicarb 27, BUN 11, creatinine 0.9, glucose\n149, white count 9.1, hematocrit 28.4, platelets 830.\nPhysical exam - Neurologically:  Alert and oriented x 3,\nnonfocal exam.  Pulmonary:  Clear to auscultation\nbilaterally.  Cardiac:  Regular rate and rhythm, S1, S2.\nSternum:  Incision with staples, clean and dry.  No erythema\nor drainage.  Abdomen was soft, nontender, nondistended with\nnormoactive bowel sounds.  Abdominal incision with staples,\nalso clean and dry.  Extremities were warm with no edema.\nRight saphenous vein graft harvest site was healing well,\nopen to air, clean and dry.\n\nCONDITION ON DISCHARGE:  Good.\n\nDISCHARGE DIAGNOSES:  Coronary artery disease, status post\ncoronary artery bypass grafting complicated by sternal\ninfection requiring sternal debridement and flap closure.\n\nDiabetes mellitus.\n\nHypercholesterolemia.\n\nGastroesophageal reflux disease.\n\nFOLLOW UP:  Follow-up with Dr. [**Last Name (STitle) 13797**] with plastic surgery\nservice in 1 week.  He is to call for an appointment at [**Telephone/Fax (1) 56789**].  He is also to have follow-up with Dr. [**Last Name (STitle) **] in 4\nweeks.  The patient is also to call for that appointment; the\nnumber is [**Telephone/Fax (1) 170**].\n\nDISCHARGE MEDICATIONS:\n1. Ranitidine 150 mg [**Hospital1 **].\n2. Simvastatin 40 mg once daily.\n3. Ferrous sulfate 325 mg once daily.\n4. Ascorbic acid 500 mg [**Hospital1 **].\n5. Zinc sulfate 220 mg once daily.\n6. Aspirin 81 mg once daily.\n7. Erythromycin ophthalmic ointment [**Hospital1 **].\n8. Colace 100 mg [**Hospital1 **].\n9. Metoprolol XL 100 mg once daily.\n10.Glargine 24 units q at bedtime.\n11.Humalog insulin sliding scale q ac and at bedtime.\n12.Lasix 20 mg once daily.\n13.Potassium chloride 20 mEq once daily.\n14.Amiodarone 400 mg [**Hospital1 **] x 1 week, then 400 mg once daily x 1\nweek, then 200 mg once daily.\n15.Oxacillin 2 grams q 4 h through [**12-28**].\n16.Warfarin as directed to maintain a target INR of 2 to 2.5.\nThe patient's warfarin doses starting with 4 days ago - 3 mg,\n5 mg, 5 mg, 5 mg.  The patient is to receive 4 mg on the [**2032-11-29**].Albuterol 2 puffs qid prn.\n\nDISPOSITION:  The patient is to be discharged to\nrehabilitation.\n\n\n\n                        [**First Name11 (Name Pattern1) **] [**Last Name (NamePattern1) 1714**], [**MD Number(1) 1715**]\n\nDictated By:[**Last Name (NamePattern4) 1718**]\nMEDQUIST36\nD:  [**2119-12-1**] 13:31:57\nT:  [**2119-12-1**] 14:15:12\nJob#:  [**Job Number 56790**]\n\n\n\n"""

In [ ]:
# from happytransformer import HappyTextClassification
# import re
# from collections import Counter, namedtuple

# # Define a simple namedtuple to represent the classification result
# TextClassificationResult = namedtuple("TextClassificationResult", ["label", "score"])

# # Load your pre-trained model and tokenizer
# model = HappyTextClassification(model_type="BioBERT", model_name="/content/drive/MyDrive/finetuned_model")
# tokenizer = model.tokenizer

# def prep_simple(pdf_read):
#     # Your existing preprocessing code
#     pdf_modified = re.sub(' +', ' ', pdf_read)
#     pdf_modified = pdf_modified.lower()
#     pdf_modified = pdf_modified.replace("\n", " ")
#     pdf_modified = re.sub(r'\[.*?\]', '', pdf_modified)
#     pdf_modified = re.sub(' +', ' ', pdf_modified)
#     return pdf_modified.strip()

# def tokenize_and_predict(text, max_words_per_chunk=55):
#     # Apply preprocessing
#     preprocessed_text = prep_simple(text)

#     # Tokenize the text into words
#     tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(preprocessed_text)))

#     # Calculate the total number of words in the text
#     total_words = len(tokens)

#     # Determine the number of full chunks and the remaining words
#     num_full_chunks = total_words // max_words_per_chunk
#     remaining_words = total_words % max_words_per_chunk

#     # Initialize predictions
#     predictions = []

#     # Process each full chunk
#     for i in range(num_full_chunks):
#         start_idx = i * max_words_per_chunk
#         end_idx = (i + 1) * max_words_per_chunk
#         chunk = " ".join(tokens[start_idx:end_idx])

#         # Make prediction for the chunk
#         prediction = model.classify_text(chunk)
#         predictions.append(prediction)

#     # Process the remaining chunk
#     if remaining_words > 0:
#         start_idx = num_full_chunks * max_words_per_chunk
#         end_idx = total_words
#         remaining_chunk = " ".join(tokens[start_idx:end_idx])

#         # Make prediction for the remaining chunk
#         prediction = model.classify_text(remaining_chunk)
#         predictions.append(prediction)

#     # Aggregate predictions (simple majority vote based on label and score)
#     aggregated_prediction = aggregate_predictions(predictions)

#     return aggregated_prediction

# def aggregate_predictions(predictions):
#     # Count occurrences of each label in the predictions
#     label_counts = Counter([result.label for result in predictions])

#     # Find the label with the maximum count
#     majority_label = max(label_counts, key=label_counts.get)

#     # Find the average score for the majority label
#     majority_score = sum([result.score for result in predictions if result.label == majority_label]) / label_counts[majority_label]

#     # Create a TextClassificationResult object for the aggregated prediction
#     aggregated_prediction = TextClassificationResult(label=majority_label, score=majority_score)

#     return aggregated_prediction

# # Example usage
# text_to_predict = "Your long text here..."
# final_prediction = tokenize_and_predict(text_to_predict)
# print(final_prediction.label)  # Access the label of the aggregated prediction
# print(final_prediction.score)  # Access the score of the aggregated prediction


In [ ]:
#Test
# test_result = model.test("/content/drive/MyDrive/processed_datasets/2_whole_Dataset_test_df_60(50)_unlabelled.csv")

In [ ]:
# print(type(test_result))
# print(test_result)
# print(type(test_result[0]))
# print(test_result[0])
# print(test_result[0].label)

In [ ]:
#classification report representing model performance
# from sklearn.metrics import classification_report

# true_labels = [example["label"] for example in test_data]
# predicted_labels = predictions.labels

# report = classification_report(true_labels, predicted_labels)

In [ ]:
# Import necessary libraries
#deployment using streamlit
import streamlit as st
from happytransformer import HappyTextClassification

# Load the pre-trained text classification model
model = HappyTextClassification.from_pretrained("/content/drive/MyDrive/finetuned_model")

# Streamlit app
st.title("Text Classification App")

# User input
user_input = st.text_area("Enter text for classification:")

# Make prediction when the user clicks the "Classify" button
if st.button("Classify"):
    # Generate prediction
    prediction = model.classify_text(user_input)

    # Display the prediction result
    st.write("Prediction:")
    st.write(f"Class: {prediction.label}")
    st.write(f"Confidence: {prediction.score:.2f}")